In [43]:
import numpy as np
import pandas as pd
from surprise import KNNBasic,evaluate
import os
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

In [44]:
reader = Reader()

training_df=pd.read_csv('datasets/train.csv')
test_df=pd.read_csv('datasets/test.csv')

users_list=pd.read_csv('datasets/metadata-users.csv')

data = Dataset.load_from_df(training_df[['User', 'Track', 'Rating']], reader)

print(type(data))
##data.split(n_folds=5)

<class 'surprise.dataset.DatasetAutoFolds'>


In [9]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate

trainset, testset = train_test_split(data, test_size=.15)

# ----- SVD ----- #

# param_grid = {'n_factors': [110, 120, 140, 160], 'n_epochs': [9, 10, 11], 'lr_all': [0.001, 0.003, 0.005, 0.008],
#               'reg_all': [0.08, 0.1, 0.15]}
# gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
# gs.fit(data)
# algo = gs.best_estimator['rmse']
# print(gs.best_score['rmse'])
# print(gs.best_params['rmse'])
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
print(type(trainset))
print(type(testset))

# Use the new parameters with the train data
algo = SVD(n_factors=160, n_epochs=10, lr_all=0.005, reg_all=0.1)
algo.fit(trainset)
test_pred = algo.test(testset)
print("SVD : Test Set")
accuracy.rmse(test_pred, verbose=True)

<class 'surprise.trainset.Trainset'>
<class 'list'>
SVD : Test Set
RMSE: 38.7505


38.75053684723277

In [ ]:
Ratings_df = training_df.pivot(index = 'User', columns ='Track', values = 'Rating').fillna(0)

In [ ]:
Ratings_df

In [ ]:
R = Ratings_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)


In [ ]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)
sigma = np.diag(sigma)


In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)




In [ ]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = Ratings_df.columns)
preds_df.head()

In [ ]:
preds_df

In [ ]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
# define a cross-validation iterator
kf = KFold(n_splits=4)
algo = SVD(n_factors=15, n_epochs=5, lr_all=0.1)

for train, test in kf.split(data):
    # train and test algorithm.
    algo.fit(train)
    predictions = algo.test(test)
    
    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [45]:
test_df

,User,Track
0,466,0
1,1173,0
2,566,1
3,49,3
4,772,3
5,1068,3
6,1534,3
7,39132,150
8,39671,150
9,40638,150


In [27]:
min(list(training_df['Rating']))

0

In [22]:
training_df

,User,Track,Rating
0,1,0,27
1,4,0,10
2,8,0,30
3,10,0,30
4,11,0,14
5,12,0,34
6,13,0,37
7,15,0,56
8,17,0,44
9,18,0,29


In [26]:
for index, row in test_df.iterrows():
    key=str(row['User'])+"-"+str(row['Track'])
    print(key)

466-0
1173-0
566-1
49-3
772-3
1068-3
1534-3
39132-150
39671-150
40638-150
40715-150
40843-150
40742-151
2902-4
2087-6
2736-6
3699-6
2132-7
4554-7
4566-7
2075-8
4648-8
4955-8
24180-69
24711-69
21545-70
21549-70
24174-70
24906-70
47507-173
47603-173
49132-174
49140-174
49123-175
50270-175
50292-175
1874-9
4858-9
2225-10
2684-10
5316-10
2501-11
4101-11
4207-11
4349-11
37779-11
2184-13
2283-13
5057-13
5365-13
6828-15
7084-16
39004-148
38847-149
39088-149
6683-17
8137-17
7885-18
8355-18
9376-20
8400-21
9591-21
8805-22
10043-22
8255-23
8570-23
5922-25
9634-25
10334-27
33857-140
34230-140
35230-140
35807-140
34337-141
35457-141
35593-141
34102-142
34364-142
35230-142
36023-142
36034-143
33230-144
34864-145
35548-145
35776-145
8055-28
9533-28
10637-28
7824-29
10141-29
39480-152
40162-152
40289-152
40829-152
39259-153
39910-153
40221-154
11144-30
11537-31
13117-31
13274-31
13409-31
13566-31
12180-32
13368-32
28140-94
28192-94
28368-94
29656-94
28698-95
29663-95
30008-96
31779-96
31843-96
11986-

28270-90
29432-90
29970-90
30221-91
30413-91
29277-92
30337-92
31281-93
31347-93
28577-97
29812-97
30613-97
32074-98
30105-99
31316-99
29250-100
31864-100
31677-101
31936-101
31940-101
30518-102
32045-102
28280-103
29317-104
30688-104
31378-104
28540-106
29841-106
30331-106
31378-106
36242-146
36448-147
37401-147
48045-176
49558-176
50572-177
50702-177
50735-177
50735-179
48136-180
49449-180
49453-180
41103-155
41215-155
41274-155
41866-156
41386-157
42144-158
42245-159
41489-162
42107-162
42600-162
42728-162
43556-163
44153-163
43573-164
44208-165
43547-166
45107-166
45336-166
44502-167
43358-168
43785-168
45099-168
45107-168
44156-169
44157-169
44284-169
44480-169
44481-169
45789-170
47111-170
45740-171
46824-171
47836-172
48009-172
50396-182
50839-182
48818-183
50568-183
50702-183
50734-183
38933-1
924-2
1429-2
40275-150
38730-151
39971-151
40883-151
2339-4
3137-4
3955-4
2235-5
2684-5
2685-5
2890-5
3744-5
3745-5
1928-6
3853-6
4060-7
5119-7
5207-8
23068-68
23173-69
24696-69
49203-173

7361-19
7672-19
8486-19
6640-25
10638-25
10639-25
9851-26
8835-27
9645-27
33591-140
34630-140
35712-140
34030-141
35025-142
34615-144
35591-144
35735-144
35849-144
33568-145
10604-28
10579-29
10773-29
10871-29
38749-152
39853-152
40315-152
40682-152
39808-153
40446-153
40810-154
12130-30
12620-31
11834-32
12206-32
29511-94
30160-94
31997-95
29248-96
31629-96
32043-96
16053-35
33678-134
34277-134
34398-134
34603-134
34490-135
34278-136
33573-137
33693-137
35026-138
33693-139
33701-139
15332-36
16730-37
18239-44
17426-45
18381-46
17212-22
17717-22
17859-22
18608-22
19842-22
19972-22
19712-47
20052-47
18232-48
19311-48
22667-48
22891-48
17909-49
19308-49
19647-50
19530-51
20271-51
21920-60
19928-52
20969-52
20167-53
20601-53
20604-53
19311-54
19964-54
20453-54
32296-54
19084-55
20601-55
32461-108
32849-108
32243-110
32351-110
33166-110
32420-112
32420-114
32857-114
32835-117
32198-118
32634-119
32521-121
32603-121
32539-123
32932-130
32216-132
32782-132
32467-133
32976-133
18236-56
18858-

24844-70
24904-70
49832-173
49552-174
50002-174
2790-9
5414-9
2673-10
2035-11
2693-11
36508-11
38286-11
2674-12
3899-12
5112-12
5298-12
5463-12
37154-12
37470-12
37829-12
5613-14
6669-15
6677-15
6976-15
39639-148
39425-149
7486-17
8613-17
6212-18
6227-18
7959-18
8803-18
7150-19
7503-19
7799-19
9009-20
9632-21
7883-22
10037-22
10275-22
10279-22
8569-23
8612-23
8920-23
9984-23
7725-24
8199-24
10785-24
10550-25
10551-25
9272-27
33593-140
34520-140
34550-140
34923-140
35425-140
35512-140
35827-140
33269-141
35883-141
34593-142
35670-142
35613-143
34837-144
35345-144
35717-144
33338-145
7855-29
8965-29
40527-152
40655-152
38443-153
38488-153
40247-153
40754-153
39495-154
40781-154
12275-30
13579-32
31174-94
31190-94
31733-94
31601-95
31776-95
30882-96
11637-33
17045-42
33586-134
33941-134
35081-135
33234-136
33241-136
34601-136
34730-136
33342-137
33613-137
33691-137
34305-137
34313-137
33242-138
34241-138
34923-138
33613-139
33691-139
16844-38
17851-44
18321-44
17751-45
18402-45
17151-46
1

17443-22
17804-22
18266-22
19560-47
20277-47
20309-47
19531-48
19604-48
22379-48
20063-49
20419-49
19160-50
19828-50
20256-50
20271-50
19470-51
20268-51
21646-60
21787-60
23038-60
20942-52
17346-53
18537-53
18758-53
19340-53
20869-53
17273-54
19528-54
20930-54
19341-55
19928-55
19968-55
20000-55
32284-108
32931-108
33012-109
33047-110
32380-113
32417-116
32682-120
32312-121
32398-124
32603-124
32462-126
32227-127
32279-127
32549-128
33038-130
32723-131
33056-132
18318-56
18686-56
21063-56
17868-57
18346-57
19755-57
21152-58
19573-59
20296-59
23288-61
22379-62
23515-62
21527-64
23577-65
23641-65
23863-65
24012-65
24159-65
21363-66
22521-66
21319-67
23537-67
23742-67
22884-71
24984-71
25220-71
23180-72
21933-73
21630-74
22785-74
21431-77
22843-77
23292-77
26949-79
27214-79
27192-80
26898-81
27651-82
26196-84
26473-85
27597-85
29065-86
27786-87
27960-87
29003-87
29209-87
28195-88
29216-88
29883-90
27730-91
29817-91
30475-91
29744-92
31044-93
31272-93
31376-97
31876-98
32130-99
31675-100
3

...

41469-162
42292-162
42619-162
42991-162
44187-163
44315-163
44292-164
44770-164
44910-164
44324-166
44416-166
44959-166
44429-167
44886-167
45114-167
43742-168
44668-169
44796-169
46398-170
50780-182
49429-183
50913-183
1732-0
39032-1
39033-1
266-2
1529-2
1480-3
40120-150
40248-150
3357-4
3860-4
3197-5
3001-6
2751-7
3090-7
3197-7
3252-7
4461-7
3302-8
3310-8
3812-8
4574-8
22352-68
22621-69
23877-69
22823-70
23540-70
24748-70
24808-70
49469-173
49597-173
49760-173
47900-175
50216-175
2589-9
3463-9
3962-10
5067-10
1998-11
5319-11
36210-11
36267-11
36776-11
2302-12
2697-12
3354-12
5075-12
5367-12
38224-12
3553-13
6076-14
6180-14
6653-16
6768-16
39094-148
39745-148
6140-17
6220-17
7390-17
8058-17
8533-17
8812-17
8828-17
7139-18
7140-18
5662-19
8517-19
9124-20
9140-22
9436-22
10103-22
7045-24
7061-24
10863-24
7083-25
9220-25
6650-26
33617-140
34063-140
35582-140
35903-140
35525-141
34124-142
34105-143
34546-143
35525-143
36004-143
33977-144
34629-144
34645-144
35081-144
35393-144
33598-145
4

29292-90
27736-91
29117-91
29794-91
30503-91
27721-93
30151-93
30361-93
28078-97
29222-98
29721-98
30421-98
31760-98
27913-101
28176-101
29637-101
29708-101
29741-101
31406-101
31289-102
31293-102
28298-103
29838-103
27882-104
29589-104
31096-104
28531-105
30986-105
31908-105
36171-147
37245-147
37503-147
48210-176
49575-176
49722-176
49849-176
49734-177
49862-177
50536-177
49106-178
49271-178
49499-178
47398-179
47371-180
48317-180
49499-180
50185-180
50804-180
48781-181
49954-181
41365-155
41582-155
42462-155
42499-157
41510-158
42392-158
43253-158
41656-159
42515-159
42627-159
42375-160
43133-160
42049-161
42081-161
42293-161
41109-162
44667-163
45194-163
43467-164
44768-164
43356-165
44840-165
44876-165
44966-165
43461-166
43695-166
44087-166
44772-166
44983-166
43521-167
44417-167
44873-167
43494-168
43502-168
44623-168
46960-171
47096-171
48318-172
48435-172
50654-182
50804-182
350-1
1414-1
39469-1
40120-1
40638-1
1254-2
1711-3
39277-150
40148-150
39652-151
40237-151
2726-4
2920-

32378-109
32743-109
32323-112
33055-112
32593-118
32448-122
32574-122
32193-124
33136-126
33010-130
17757-56
20717-56
18349-57
18931-57
19342-57
20084-57
20673-57
17647-58
18121-58
19707-58
19708-58
20932-58
17713-59
18461-59
19440-59
19574-59
20374-59
23476-61
21964-62
23518-62
23586-62
22130-64
23688-64
22130-66
23099-66
21609-67
23958-67
22114-71
22445-75
24371-75
23632-76
23749-77
21992-78
22178-78
23340-78
25081-78
27461-79
26261-80
26905-80
27074-85
27318-85
33929-85
35256-85
35866-85
27917-87
28105-87
28520-87
29838-88
29979-88
29728-90
30379-90
29537-92
30254-92
30587-92
30597-92
27966-93
29297-93
30144-93
30437-93
30958-93
31107-93
28335-97
29214-98
30355-99
31197-99
31578-99
31857-100
32038-100
29511-101
29982-101
29733-103
31224-103
29606-104
27670-105
29915-105
31742-105
38211-146
36254-147
38238-147
38254-147
47364-176
50620-176
47631-177
48761-177
50411-177
50539-177
47364-178
48031-178
48234-178
47584-179
48866-180
50587-180
41583-156
41785-156
41913-156
41316-157
41579-

20546-55
32808-115
33141-115
32754-117
32845-117
32876-117
32334-118
32977-118
32832-120
32986-121
32415-122
32536-124
32662-124
33050-124
18514-56
20951-57
20486-58
18804-59
21546-61
22243-61
22384-61
23520-61
24046-61
23582-62
24055-62
23379-63
23856-63
24164-63
23183-64
23210-64
23880-65
23229-67
21307-71
23270-71
24047-71
24083-71
25255-71
24440-72
21650-73
24285-74
22097-75
23735-75
24744-75
21749-76
22553-78
26361-79
26462-80
26478-80
27226-80
25533-81
25855-81
26286-82
25517-83
25533-83
26391-85
26659-85
26663-85
33406-85
34148-85
34470-85
35285-85
29498-86
28185-87
29093-89
30032-89
30509-89
29951-90
29715-91
27719-92
28740-92
28756-92
30666-93
31334-97
31691-97
31762-97
27915-98
29708-98
32179-98
30598-99
30997-99
28261-101
29375-101
29567-101
30258-101
30627-101
30691-101
29861-102
28776-103
29493-103
27705-104
27748-104
29121-104
29979-104
31708-104
31026-105
31098-105
31256-105
31448-105
31473-105
36691-146
37082-146
36574-147
37801-147
47716-176
47791-176
50400-176
50611-1

34064-137
34201-137
34828-137
33983-138
34111-138
33986-139
14969-36
17692-44
18016-44
18083-44
17274-45
17824-45
17141-46
17984-46
18610-22
18977-22
19958-22
17124-47
17683-47
18496-47
19408-48
22494-48
22812-48
18275-49
19022-49
19614-50
20373-50
19411-51
18380-52
19930-52
18655-53
19715-53
20414-53
17987-54
18460-54
18981-54
19970-54
20687-54
18856-55
19715-55
20492-55
33182-111
33178-113
32267-114
32938-114
32746-115
32896-121
32960-121
32664-123
17129-56
17541-56
19601-56
19602-56
20257-57
21124-57
21125-57
18120-59
18501-59
18642-59
19256-59
21080-59
21112-59
22380-61
22004-62
23626-62
23843-62
23585-63
22582-64
23800-64
24051-64
22585-65
23458-65
24137-65
21818-66
22582-66
23295-67
23402-67
24137-67
24192-67
24256-67
24568-71
22949-72
23804-72
24788-72
22493-73
24355-73
25181-73
23890-74
24724-74
25383-74
23646-75
25330-75
23519-76
21999-77
23789-77
24091-78
24656-78
25747-79
27009-79
26128-80
26336-80
26868-80
25540-81
25556-81
26973-81
26336-82
27279-82
26560-83
26606-83
35159

In [32]:
test_df=pd.read_csv('datasets/test.csv',sep=",",skiprows=0)


In [33]:
test_df

,User,Track
0,466,0
1,1173,0
2,566,1
3,49,3
4,772,3
5,1068,3
6,1534,3
7,39132,150
8,39671,150
9,40638,150


In [36]:
dict={}

dict['3423-100']=23
dict['3423-1020']=2
dict['3423-1010']=13
dict['3423-1020']=43





In [41]:

df=pd.DataFrame(list(dict.items()),columns=['Date', 'DateValue'])

In [42]:
df

,Date,DateValue
0,3423-100,23
1,3423-1020,43
2,3423-1010,13
